In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.options.display.max_rows = 999
from tqdm import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

In [ ]:
#The adjustment factor is 5.  Look at the change in the price and volume columns to see the impact of the stock split.
data[data["SecuritiesCode"]==8057].iloc[7:10]

In [ ]:
#This part of the code maps the Date to an integer
x = np.array(data.loc[data["SecuritiesCode"]==1301,"Date"])
di={x[i]:i for i in range(x.shape[0])}
data["DateInt"] = data["Date"]
data.replace({"DateInt": di}, inplace=True)

#This code loops through each security
securities = np.array(data.loc[data["AdjustmentFactor"]!=1,"SecuritiesCode"])
for s in tqdm(securities):
    #chooses the rows where the adjustment factor isn't 1
    indexes = s==data.SecuritiesCode
    d = data[indexes]
    splits = d.loc[d["AdjustmentFactor"]!=1,["AdjustmentFactor","DateInt"]]
    for i in splits.iterrows():
        #Chooses all rows before and at the factor and multiplies prices by the factor and divides volume by the factor
        index = d[d.DateInt<=i[1].DateInt].index
        data.loc[index,["Close","Low","High","Open"]] = data.loc[index,["Close","Low","High","Open"]] * i[1].AdjustmentFactor
        data.loc[index,"Volume"] = data.loc[index,"Volume"] / i[1].AdjustmentFactor

In [ ]:
#After running the above code, the price and volume are comparable before and after the split
data[data["SecuritiesCode"]==8057].iloc[7:10]